In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [3]:
cap = cv2.VideoCapture('./data/vtest.avi')
if not cap.isOpened():
    print('Error~')
    
height, width = (int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)), int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)))

bkg_gray = cv2.imread('./data/avg_gray.png', cv2.IMREAD_GRAYSCALE)
bkg_bgr = cv2.imread('./data/avg_bgr.png')

In [4]:
TH = 80 #밝기값 임계값
AREA_TH = 100
mode = cv2.RETR_EXTERNAL
method = cv2.CHAIN_APPROX_SIMPLE

In [5]:
t = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    t+=1
    #gray
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    diff_gray = cv2.absdiff(gray, bkg_gray)
    #color
    diff_bgr = cv2.absdiff(frame, bkg_bgr)
    db, dg, dr = cv2.split(diff_bgr)
    ret, bb = cv2.threshold(db, TH, 255, cv2.THRESH_BINARY)
    ret, bg = cv2.threshold(dg, TH, 255, cv2.THRESH_BINARY)
    ret, br = cv2.threshold(dr, TH, 255, cv2.THRESH_BINARY)
    
    bimage = cv2.bitwise_or(bb, bg) #총합
    bimage = cv2.bitwise_or(br, bimage)
    
    bimage = cv2.erode(bimage, None, 5) #모폴로지 적용
    bimage = cv2.dilate(bimage, None, 5)
    bimage = cv2.erode(bimage, None, 7)
    
    contours, hierarcy = cv2.findContours(bimage, mode, method)
    cv2.drawContours(frame, contours, -1, (255, 0, 0), 1)
    for i, cnt in enumerate(contours):
        area = cv2.contourArea(cnt)
        if area > AREA_TH:
            x, y, w, h = cv2.boundingRect(cnt)
            cv2.rectangle(frame, (x, y), (x+w, x+h), (0, 0, 255), 2)
            
    cv2.imshow('frame', frame)
    cv2.imshow('bimage', bimage)
    cv2.imshow('diff_gray', diff_gray)
    cv2.imshow('diff_bgr', diff_bgr)
    
            
if cap.isOpened():
    cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.2.0) C:\projects\opencv-python\opencv\modules\core\src\arithm.cpp:669: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'cv::arithm_op'
